# Homework #3 - Rossman Store Sales

Group Members:

Hoo De Lin (A0126576X)

Yap Han Chiang (A0125168E)

# Analyzing the data:

Before we try figuring out which model to fit in, we need to understand the data, and clean/modify the data to be able to feed into any model

The feature that I believe are quite important are:

- Competition
- Holidays
- Type of stores
- Type of day (Monday - Sunday)
- Having promotions
- Promo2 since

Interesting to note is that, 

In [1]:
import numpy as np
import pandas as pd
import math
from datetime import datetime
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as pl
%matplotlib inline
import seaborn as sb 
import csv
from sklearn.ensemble import RandomForestRegressor

# import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [155]:
trainData = pd.read_csv('train_v2.csv', parse_dates=['Date'], dtype={'StateHoliday': 'str'})
storeData = pd.read_csv('store.csv')
testData = pd.read_csv('test_v2.csv', parse_dates=['Date'], dtype={'StateHoliday': 'str', 'StoreType': 'str', 'Assortment': 'str'})
trainData = pd.merge(storeData, trainData, on='Store')
testData = testData.reset_index().merge(storeData,on='Store')
index = testData['index']
testData.drop('index', axis=1, inplace=True)

In [156]:
testData

,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
5,1,7,2015-07-26,0,0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
6,1,6,2015-07-25,500,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
7,1,5,2015-07-24,459,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
8,1,4,2015-07-23,503,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
9,1,3,2015-07-22,463,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [157]:
# Cleaning the data
competition_distance_mean = trainData['CompetitionDistance'].mean()
customer_mean = trainData['Customers'].mean()
competition_distance_std = trainData['CompetitionDistance'].std()
customer_std = trainData['Customers'].std()

def normalizeDistance(row):
    return (row['CompetitionDistance'] - competition_distance_mean) / competition_distance_std
def normalizeCustomer(row):
    return (row['Customers'] - customer_mean) / customer_std
def hasHoliday(row):
    if row['StateHoliday'] != "0" or row['SchoolHoliday'] != "0":
        return 1
    else:
        return 0
def calculateCompetitionAge(row):
        if math.isnan(row['CompetitionOpenSinceMonth']):
            return 0
        else:
            f_date = datetime(int(row['CompetitionOpenSinceYear']), int(row['CompetitionOpenSinceMonth']), 1, 0, 0, 0)
            return math.floor(((row['Date'].to_pydatetime() - f_date).days) / 30)

month_num_name_dict = {
    "1": "Jan",
    "2": "Feb",
    "3": "Mar",
    "4": "Apr",
    "5": "May",
    "6": "Jun",
    "7": "Jul",
    "8": "Aug",
    "9": "Sept",
    "10": "Oct",
    "11": "Nov",
    "12": "Dec"
}

def isHavingPromo2(row):
    if not isinstance(row['PromoInterval'], str):
        return 0
    elif month_num_name_dict[str(row['Date'].to_pydatetime().month)] in row['PromoInterval']:
        return 1
    else:
        return 0

# Possible thought: If we are not at the promo interval, should promo2Age have any effect?
def promo2Age(row):
    if row['Promo2'] == 0:
        return 0
    else:
#        https://stackoverflow.com/a/17087427
        date = str(int(row["Promo2SinceYear"])) + "-W" + str(int(row["Promo2SinceWeek"]))
        f_date = datetime.strptime(date + '-0', "%Y-W%W-%w")
        return math.floor((row['Date'].to_pydatetime() - f_date).days/ 30)
mapCharToInt = {'0': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4}
mapPromoIntervalToInt = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7,
                        'Aug': 8, 'Sept': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}


In [158]:
trainData['HasHoliday'] = trainData.apply(lambda row: hasHoliday(row), axis=1)
trainData['CompetitionAge'] = trainData.apply(lambda row: calculateCompetitionAge(row), axis=1)
trainData['HavingPromo2'] = trainData.apply(lambda row: isHavingPromo2(row), axis=1)
trainData['Promo2Age'] = trainData.apply(lambda row: promo2Age(row), axis=1)
trainData['CompetitionDistance'] = trainData.apply(lambda row: normalizeDistance(row), axis=1)
trainData['Customers'] = trainData.apply(lambda row: normalizeCustomer(row), axis=1)
trainData.replace(mapCharToInt, inplace=True)
trainData.fillna(0, inplace=True)
testData['HasHoliday'] = testData.apply(lambda row: hasHoliday(row), axis=1)
testData['CompetitionAge'] = testData.apply(lambda row: calculateCompetitionAge(row), axis=1)
testData['HavingPromo2'] = testData.apply(lambda row: isHavingPromo2(row), axis=1)
testData['Promo2Age'] = testData.apply(lambda row: promo2Age(row), axis=1)
testData['CompetitionDistance'] = testData.apply(lambda row: normalizeDistance(row), axis=1)
testData['Customers'] = testData.apply(lambda row: normalizeCustomer(row), axis=1)
testData.replace(mapCharToInt, inplace=True)
testData.fillna(0, inplace=True)

In [159]:
trainData

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,HasHoliday,CompetitionAge,HavingPromo2,Promo2Age
0,1,3,1,-0.538119,9.0,2008.0,0,0.0,0.0,0,...,5577,-0.091497,1,1,0,0,1,78,0,0
1,1,3,1,-0.538119,9.0,2008.0,0,0.0,0.0,0,...,5171,-0.177943,1,1,0,0,1,78,0,0
2,1,3,1,-0.538119,9.0,2008.0,0,0.0,0.0,0,...,4310,-0.250317,1,1,0,0,1,78,0,0
3,1,3,1,-0.538119,9.0,2008.0,0,0.0,0.0,0,...,4806,-0.260369,1,1,0,0,1,77,0,0
4,1,3,1,-0.538119,9.0,2008.0,0,0.0,0.0,0,...,4781,-0.192016,1,1,0,0,1,77,0,0
5,1,3,1,-0.538119,9.0,2008.0,0,0.0,0.0,0,...,0,-1.329891,0,0,0,0,1,77,0,0
6,1,3,1,-0.538119,9.0,2008.0,0,0.0,0.0,0,...,4840,-0.123663,1,0,0,0,1,77,0,0
7,1,3,1,-0.538119,9.0,2008.0,0,0.0,0.0,0,...,3586,-0.346815,1,0,0,0,1,77,0,0
8,1,3,1,-0.538119,9.0,2008.0,0,0.0,0.0,0,...,3492,-0.338774,1,0,0,0,1,77,0,0
9,1,3,1,-0.538119,9.0,2008.0,0,0.0,0.0,0,...,3299,-0.405116,1,0,0,0,1,77,0,0


In [160]:
testData

,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,HasHoliday,CompetitionAge,HavingPromo2,Promo2Age
0,1,5,2015-07-31,-0.214130,1,1,0,1,3,1,...,9.0,2008.0,0,0.0,0.0,0,1,84,0,0
1,1,4,2015-07-30,-0.232224,1,1,0,1,3,1,...,9.0,2008.0,0,0.0,0.0,0,1,84,0,0
2,1,3,2015-07-29,-0.278462,1,1,0,1,3,1,...,9.0,2008.0,0,0.0,0.0,0,1,84,0,0
3,1,2,2015-07-28,-0.204078,1,1,0,1,3,1,...,9.0,2008.0,0,0.0,0.0,0,1,84,0,0
4,1,1,2015-07-27,-0.099538,1,1,0,1,3,1,...,9.0,2008.0,0,0.0,0.0,0,1,84,0,0
5,1,7,2015-07-26,-1.329891,0,0,0,0,3,1,...,9.0,2008.0,0,0.0,0.0,0,1,83,0,0
6,1,6,2015-07-25,-0.324701,1,0,0,0,3,1,...,9.0,2008.0,0,0.0,0.0,0,1,83,0,0
7,1,5,2015-07-24,-0.407127,1,0,0,0,3,1,...,9.0,2008.0,0,0.0,0.0,0,1,83,0,0
8,1,4,2015-07-23,-0.318670,1,0,0,0,3,1,...,9.0,2008.0,0,0.0,0.0,0,1,83,0,0
9,1,3,2015-07-22,-0.399085,1,0,0,0,3,1,...,9.0,2008.0,0,0.0,0.0,0,1,83,0,0


In [161]:
# Dropping features
_trainData = trainData.drop(['Date','CompetitionOpenSinceMonth','CompetitionOpenSinceYear','StateHoliday','SchoolHoliday','Promo2SinceWeek', 'Promo2SinceYear','PromoInterval', 'Promo2', 'Open'], axis=1)
_testData = testData.drop(['Date','CompetitionOpenSinceMonth','CompetitionOpenSinceYear','StateHoliday','SchoolHoliday','Promo2SinceWeek', 'Promo2SinceYear','PromoInterval', 'Promo2', 'Open'], axis=1)
y = _trainData[_trainData.Customers != 0].Sales.values
_trainData = _trainData[_trainData.Customers != 0].drop(['Sales', 'Store'], axis=1)
_testData = _testData.drop('Store', axis=1)

In [162]:
_trainData

,StoreType,Assortment,CompetitionDistance,DayOfWeek,Customers,Promo,HasHoliday,CompetitionAge,HavingPromo2,Promo2Age
0,3,1,-0.538119,5,-0.091497,1,1,78,0,0
1,3,1,-0.538119,4,-0.177943,1,1,78,0,0
2,3,1,-0.538119,3,-0.250317,1,1,78,0,0
3,3,1,-0.538119,2,-0.260369,1,1,77,0,0
4,3,1,-0.538119,1,-0.192016,1,1,77,0,0
5,3,1,-0.538119,7,-1.329891,0,1,77,0,0
6,3,1,-0.538119,6,-0.123663,0,1,77,0,0
7,3,1,-0.538119,5,-0.346815,0,1,77,0,0
8,3,1,-0.538119,4,-0.338774,0,1,77,0,0
9,3,1,-0.538119,3,-0.405116,0,1,77,0,0


In [163]:
_testData

,DayOfWeek,Customers,Promo,StoreType,Assortment,CompetitionDistance,HasHoliday,CompetitionAge,HavingPromo2,Promo2Age
0,5,-0.214130,1,3,1,-0.538119,1,84,0,0
1,4,-0.232224,1,3,1,-0.538119,1,84,0,0
2,3,-0.278462,1,3,1,-0.538119,1,84,0,0
3,2,-0.204078,1,3,1,-0.538119,1,84,0,0
4,1,-0.099538,1,3,1,-0.538119,1,84,0,0
5,7,-1.329891,0,3,1,-0.538119,1,83,0,0
6,6,-0.324701,0,3,1,-0.538119,1,83,0,0
7,5,-0.407127,0,3,1,-0.538119,1,83,0,0
8,4,-0.318670,0,3,1,-0.538119,1,83,0,0
9,3,-0.399085,0,3,1,-0.538119,1,83,0,0


In [168]:
X = _trainData.values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
rfr = RandomForestRegressor()
clf = rfr.fit(X_train, y_train)
print(clf.score(X_val, y_val))
rfr.fit(X,y)
print(rfr.feature_importances_)

0.954804764762
[ 0.02883881  0.01306917  0.03734832  0.00541607  0.87852298  0.01059308
  0.          0.01347933  0.00113394  0.01159829]


In [169]:
results = rfr.predict(_testData.as_matrix())
store_closed_index = np.where(_testData.Customers == (0 - customer_mean)/customer_std)[0]

print(store_closed_index)

for val in store_closed_index:
    results[val] = 0
with open('submission.csv', 'w') as csvfile:
    fieldnames = ['Id', 'Sales']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for idx, val in enumerate(results):
        writer.writerow({'Id': index[idx] + 1, 'Sales': val})
    csvfile.close()

[    5    12    19 ..., 33432 33439 33446]


How bout some xgboost?